```
         Copyright Rein Halbersma 2018-2021.
Distributed under the Boost Software License, Version 1.0.
   (See accompanying file LICENSE_1_0.txt or copy at
         http://www.boost.org/LICENSE_1_0.txt)
```

# Best Winning Percentage Stratego Game Board Setup on Gravon

## 1. Reproducing Dobby125's [blog post](http://stratego-tips.blogspot.com/2017/05/best-winning-percentage-stratego-game.html)

In [1]:
from IPython.display import display

import gravon.package as pkg
import gravon.pattern as pattern

db_setups = pkg.load_dataset('db_setups')

In [2]:
best_setups = {
    'v1': (
        """
        2465276146
        7282229X8B
        25347263BF
        53BBB4353B
        """
    ),
    'v2': (
        """
        2545274146
        7298242X8B
        52362273BF
        35BB6B363B
        """
    ),
}

In [3]:
classic_xml = (db_setups
    .query('type == "classic" & ext == ".xml"')
    .assign(result = lambda r: r.result.cat.reorder_categories(['win', 'loss', 'draw']))
)

In [4]:
for s in best_setups.values():
    df = pattern.equal(classic_xml, s, mirrored=False)
    print(s)
    display(df
        .groupby(lambda _: '')
        .agg(**{
            'Wins'        : ('win',    'sum'  ), 
            'Losses'      : ('loss',   'sum'  ), 
            'Ties'        : ('draw',   'sum'  ),
            'Games Played': ('result', 'count')
        })
        .assign(**{
            'Winning Percentage': lambda r: r.Wins / (r.Wins + r.Losses)
        })
        .style.format({
            'Winning Percentage': '{:.2%}'
        })
    )
    print("Here's a listing of the game files for the setup above:\n")    
    for row in df.sort_values('filename').itertuples():
        print(f'{row.player.capitalize():4} {row.result.capitalize():4} {row.filename}') 


        2465276146
        7282229X8B
        25347263BF
        53BBB4353B
        


,Wins,Losses,Ties,Games Played,Winning Percentage
,87,9,1,97,90.62%


Here's a listing of the game files for the setup above:

Red  Loss classic-2006.0-1129.xml
Blue Win  classic-2006.0-1169.xml
Red  Loss classic-2006.0-131.xml
Blue Win  classic-2006.0-1565.xml
Red  Win  classic-2006.0-1621.xml
Blue Win  classic-2006.0-164.xml
Blue Win  classic-2006.0-1653.xml
Blue Win  classic-2006.0-1688.xml
Blue Win  classic-2006.0-1770.xml
Red  Win  classic-2006.0-1825.xml
Blue Loss classic-2006.0-1962.xml
Red  Loss classic-2006.0-212.xml
Blue Win  classic-2006.0-2446.xml
Red  Win  classic-2006.0-2464.xml
Red  Win  classic-2006.0-2501.xml
Red  Win  classic-2006.0-2675.xml
Blue Win  classic-2006.0-2869.xml
Blue Win  classic-2006.0-317.xml
Red  Win  classic-2006.0-3212.xml
Blue Win  classic-2006.0-3451.xml
Blue Win  classic-2006.0-3549.xml
Red  Win  classic-2006.0-3590.xml
Red  Win  classic-2006.0-3622.xml
Blue Win  classic-2006.0-3880.xml
Red  Win  classic-2006.0-4155.xml
Red  Win  classic-2006.0-4260.xml
Red  Win  classic-2006.0-4311.xml
Blue Win  classic-2006.0-4385

,Wins,Losses,Ties,Games Played,Winning Percentage
,45,6,1,52,88.24%


Here's a listing of the game files for the setup above:

Red  Win  classic-2005.10-1089.xml
Blue Win  classic-2005.10-1093.xml
Blue Win  classic-2005.10-1198.xml
Red  Win  classic-2005.10-1295.xml
Red  Win  classic-2005.10-131.xml
Blue Win  classic-2005.10-1407.xml
Blue Win  classic-2005.10-1428.xml
Blue Win  classic-2005.10-1575.xml
Blue Win  classic-2005.10-1604.xml
Blue Loss classic-2005.10-169.xml
Blue Win  classic-2005.10-1714.xml
Red  Win  classic-2005.10-1896.xml
Red  Win  classic-2005.10-2210.xml
Blue Win  classic-2005.10-2704.xml
Red  Win  classic-2005.10-284.xml
Blue Win  classic-2005.10-3339.xml
Blue Win  classic-2005.10-3539.xml
Red  Loss classic-2005.10-3812.xml
Red  Win  classic-2005.10-3878.xml
Blue Win  classic-2005.10-410.xml
Red  Win  classic-2005.10-4152.xml
Red  Win  classic-2005.10-4162.xml
Blue Win  classic-2005.10-4429.xml
Blue Win  classic-2005.10-4543.xml
Blue Win  classic-2005.10-4721.xml
Red  Loss classic-2005.10-4736.xml
Red  Win  classic-2005.10-4902.xml
Bl

## 2. Improvements

We make the following improvements: 
- First, we include all Gravon games, both in the .gsn and .xml formats. This increases the number of setups to over a hundred thousand.
- Second, we also include mirrored setup matches.
- Third, we order game outcomes as Win, Draw, Loss.
- Fourth, we summarize the average percentage score (win = 1.0, draw = 0.5 and loss = 0.0 points).
- Fifth, we also show statistics by the match type, the player's color and the period in which the games were played.
- Finally, we archive the various games in .zip files viewable with the [Gravon game player](http://www.gravon.de/gravon/stratego/strados2.jsp).

Note that the table cells in this notebook are not correctly aligned when rendered on GitHub.

In [ ]:
import gravon.archive as archive
from gravon.stats import league_table, score_pct

In [6]:
classic = db_setups.query('type == "classic"')

In [7]:
for version, s in best_setups.items():
    df = pattern.equal(classic, s, mirrored=True)
    print(s)
    display(df
        .groupby(lambda _: '')
        .agg(**league_table)
        .style.format(score_pct)
    )
    display(df
        .groupby('match_type')
        .agg(**league_table)
        .style.format(score_pct)
    )
    display(df
        .groupby('player')
        .agg(**league_table)
        .style.format(score_pct)
    )
    display(df
        .groupby('period')
        .agg(**league_table)
        .style.format(score_pct)
    )
    print("Here's a listing of the game files for the setup above:\n")    
    for row in df.itertuples():
        print(f'{row.player.capitalize():4} {row.result.capitalize():4} {row.filename}') 
    archive.make_zip(df.filename.tolist(), f'best_setup_{version}')


        2465276146
        7282229X8B
        25347263BF
        53BBB4353B
        


,Games,Wins,Draws,Losses,Score
,97,87,1,9,90.21%


,Games,Wins,Draws,Losses,Score
match_type,,,,,
identity,97,87,1,9,90.21%


,Games,Wins,Draws,Losses,Score
player,,,,,
red,41,37,0,4,90.24%
blue,56,50,1,5,90.18%


,Games,Wins,Draws,Losses,Score
period,,,,,
2005-12,46,40,0,6,86.96%
2006-01,19,19,0,0,100.00%
2006-02,17,16,0,1,94.12%
2006-05,13,11,0,2,84.62%
2006-07,2,1,1,0,75.00%


...

,Games,Wins,Draws,Losses,Score
,52,45,1,6,87.50%


,Games,Wins,Draws,Losses,Score
match_type,,,,,
identity,52,45,1,6,87.50%


,Games,Wins,Draws,Losses,Score
player,,,,,
red,26,21,0,5,80.77%
blue,26,24,1,1,94.23%


,Games,Wins,Draws,Losses,Score
period,,,,,
2005-10,40,34,1,5,86.25%
2005-11,10,9,0,1,90.00%
2006-01,1,1,0,0,100.00%
2006-05,1,1,0,0,100.00%


Here's a listing of the game files for the setup above:

Red  Win  classic-2005.10-131.xml
Blue Loss classic-2005.10-169.xml
Red  Win  classic-2005.10-284.xml
Blue Win  classic-2005.10-410.xml
Blue Draw classic-2005.10-660.xml
Blue Win  classic-2005.10-856.xml
Blue Win  classic-2005.10-902.xml
Red  Win  classic-2005.10-1089.xml
Blue Win  classic-2005.10-1093.xml
Blue Win  classic-2005.10-1198.xml
Red  Win  classic-2005.10-1295.xml
Blue Win  classic-2005.10-1407.xml
Blue Win  classic-2005.10-1428.xml
Blue Win  classic-2005.10-1575.xml
Blue Win  classic-2005.10-1604.xml
Blue Win  classic-2005.10-1714.xml
Red  Win  classic-2005.10-1896.xml
Red  Win  classic-2005.10-2210.xml
Blue Win  classic-2005.10-2704.xml
Blue Win  classic-2005.10-3339.xml
Blue Win  classic-2005.10-3539.xml
Red  Loss classic-2005.10-3812.xml
Red  Win  classic-2005.10-3878.xml
Red  Win  classic-2005.10-4152.xml
Red  Win  classic-2005.10-4162.xml
Blue Win  classic-2005.10-4429.xml
Blue Win  classic-2005.10-4543.xml
Blue 